In [ ]:
import tensorflow.compat.v1 as tf
import numpy as np
import os
import tensorflow_addons as tfa
from PIL import Image as PILImage

tf.disable_v2_behavior()
tf.disable_eager_execution()

# Param Choice

**Note your input should crop and resize to 372*540**

In [ ]:
#parameters
lfsize = [372, 540, 8, 8] #dimensions of light fields
batchsize = 1 
disp_mult = 4.0 #max disparity between adjacent veiws
checkpoint = 'checkpoints/model.ckpt-499' #path to store checkpoints
input_dir = 'input'# input dir
outdir = "output/" # output dir

# Model Architecture

In [ ]:
#functions for CNN layers

def weight_variable(w_shape):
    return tf.get_variable('weights', w_shape, initializer=tf.keras.initializers.glorot_normal())

def bias_variable(b_shape, init_bias=0.0):
    return tf.get_variable('bias', b_shape, initializer=tf.constant_initializer(init_bias))

def cnn_layer(input_tensor, w_shape, b_shape, layer_name, rate=1, ds=1):
    with tf.variable_scope(layer_name):
        W = weight_variable(w_shape)
        pad_amt_0 = rate * (w_shape[0] - 1)//2
        pad_amt_1 = rate * (w_shape[1] - 1)//2
        input_tensor = tf.pad(input_tensor, [[0,0],[pad_amt_0,pad_amt_0],[pad_amt_1,pad_amt_1],[0,0]], mode='SYMMETRIC')
        h = tf.nn.convolution(input_tensor, W, strides=[ds, ds], padding='VALID', dilation_rate=[rate, rate], name=layer_name + '_conv')
        h = tfa.layers.InstanceNormalization()(h + bias_variable(b_shape))
        h = tf.nn.leaky_relu(h)
        return h
    
def cnn_layer_plain(input_tensor, w_shape, b_shape, layer_name, rate=1, ds=1):
    with tf.variable_scope(layer_name):
        W = weight_variable(w_shape)
        pad_amt_0 = rate * (w_shape[0] - 1)//2
        pad_amt_1 = rate * (w_shape[1] - 1)//2
        input_tensor = tf.pad(input_tensor, [[0,0],[pad_amt_0,pad_amt_0],[pad_amt_1,pad_amt_1],[0,0]], mode='SYMMETRIC')
        h = tf.nn.convolution(input_tensor, W, strides=[ds, ds], padding='VALID', dilation_rate=[rate, rate], name=layer_name + '_conv')
        h = h + bias_variable(b_shape)
        return h
    
def cnn_layer_3D(input_tensor, w_shape, b_shape, layer_name, rate=1, ds=1):
    with tf.variable_scope(layer_name):
        W = weight_variable(w_shape)
        pad_amt_0 = rate * (w_shape[0] - 1)//2
        pad_amt_1 = rate * (w_shape[1] - 1)//2
        pad_amt_2 = rate * (w_shape[2] - 1)//2
        input_tensor = tf.pad(input_tensor, [[0,0],[pad_amt_0,pad_amt_0],[pad_amt_1,pad_amt_1],[pad_amt_2,pad_amt_2],[0,0]], mode='SYMMETRIC')
        h = tf.nn.convolution(input_tensor, W, strides=[ds, ds, ds], padding='VALID', dilation_rate=[rate, rate, rate], name=layer_name + '_conv')
        h = tfa.layers.InstanceNormalization()(h + bias_variable(b_shape))
        h = tf.nn.leaky_relu(h)
        return h
    
def cnn_layer_3D_plain(input_tensor, w_shape, b_shape, layer_name, rate=1, ds=1):
    with tf.variable_scope(layer_name):
        W = weight_variable(w_shape)
        pad_amt_0 = rate * (w_shape[0] - 1)//2
        pad_amt_1 = rate * (w_shape[1] - 1)//2
        pad_amt_2 = rate * (w_shape[2] - 1)//2
        input_tensor = tf.pad(input_tensor, [[0,0],[pad_amt_0,pad_amt_0],[pad_amt_1,pad_amt_1],[pad_amt_2,pad_amt_2],[0,0]], mode='SYMMETRIC')
        h = tf.nn.convolution(input_tensor, W, strides=[ds, ds, ds], padding='VALID', dilation_rate=[rate, rate, rate], name=layer_name + '_conv')
        h = h + bias_variable(b_shape)
        return h

In [ ]:
#network to predict ray depths from input image

def depth_network(x, lfsize, disp_mult, name):
    with tf.variable_scope(name):
        
        b_sz = tf.shape(x)[0]
        y_sz = tf.shape(x)[1]
        x_sz = tf.shape(x)[2]
        v_sz = lfsize[2]
        u_sz = lfsize[3]
        
        c1 = cnn_layer(x, [3, 3, 3, 16], [16], 'c1')
        c2 = cnn_layer(c1, [3, 3, 16, 64], [64], 'c2')
        c3 = cnn_layer(c2, [3, 3, 64, 128], [128], 'c3')
        c4 = cnn_layer(c3, [3, 3, 128, 128], [128], 'c4', rate=2)
        c5 = cnn_layer(c4, [3, 3, 128, 128], [128], 'c5', rate=4)
        c6 = cnn_layer(c5, [3, 3, 128, 128], [128], 'c6', rate=8)
        c7 = cnn_layer(c6, [3, 3, 128, 128], [128], 'c7', rate=16)
        c8 = cnn_layer(c7, [3, 3, 128, 128], [128], 'c8')
        c9 = cnn_layer(c8, [3, 3, 128, lfsize[2]*lfsize[3]], [lfsize[2]*lfsize[3]], 'c9')
        c10 = disp_mult*tf.tanh(cnn_layer_plain(c9, [3, 3, lfsize[2]*lfsize[3], lfsize[2]*lfsize[3]], \
                                                [lfsize[2]*lfsize[3]], 'c10'))
        
        return tf.reshape(c10, [b_sz, y_sz, x_sz, v_sz, u_sz])

In [ ]:
#network for refining Lambertian light field (predict occluded rays and non-Lambertian effects)

def occlusions_network(x, shear, lfsize, name):
    with tf.variable_scope(name):
        
        b_sz = tf.shape(x)[0]
        y_sz = tf.shape(x)[1]
        x_sz = tf.shape(x)[2]
        v_sz = lfsize[2]
        u_sz = lfsize[3]
        
        x = tf.transpose(tf.reshape(tf.transpose(x, perm=[0, 5, 1, 2, 3, 4]), \
                                    [b_sz, 4, y_sz, x_sz, u_sz*v_sz]), perm=[0, 4, 2, 3, 1])
        
        c1 = cnn_layer_3D(x, [3, 3, 3, 4, 8], [8], 'c1')
        c2 = cnn_layer_3D(c1, [3, 3, 3, 8, 8], [8], 'c2')
        c3 = cnn_layer_3D(c2, [3, 3, 3, 8, 8], [8], 'c3')
        c4 = cnn_layer_3D(c3, [3, 3, 3, 8, 8], [8], 'c4')
        c5 = tf.tanh(cnn_layer_3D_plain(c4, [3, 3, 3, 8, 3], [3], 'c5'))
        
        output = tf.transpose(tf.reshape(tf.transpose(c5, perm=[0, 4, 2, 3, 1]), \
                                         [b_sz, 3, y_sz, x_sz, v_sz, u_sz]), perm=[0, 2, 3, 4, 5, 1]) + shear
                
        return output

In [ ]:
#full forward model

def forward_model(x, lfsize, disp_mult):
    with tf.variable_scope('forward_model', reuse=tf.AUTO_REUSE) as scope:
        #predict ray depths from input image
        ray_depths = depth_network(x, lfsize, disp_mult, 'ray_depths')
        #shear input image by predicted ray depths to render Lambertian light field
        lf_shear_r = depth_rendering(x[:, :, :, 0], ray_depths, lfsize)
        lf_shear_g = depth_rendering(x[:, :, :, 1], ray_depths, lfsize)
        lf_shear_b = depth_rendering(x[:, :, :, 2], ray_depths, lfsize)
        lf_shear = tf.stack([lf_shear_r, lf_shear_g, lf_shear_b], axis=5)
        #occlusion/non-Lambertian prediction network
        shear_and_depth = tf.stack([lf_shear_r, lf_shear_g, lf_shear_b, tf.stop_gradient(ray_depths)], axis=5)
        y = occlusions_network(shear_and_depth, lf_shear, lfsize, 'occlusions')
        return ray_depths, lf_shear, y

In [ ]:
#render light field from input image and ray depths

def depth_rendering(central, ray_depths, lfsize):
    with tf.variable_scope('depth_rendering') as scope:
        b_sz = tf.shape(central)[0]
        y_sz = tf.shape(central)[1]
        x_sz = tf.shape(central)[2]
        u_sz = lfsize[2]
        v_sz = lfsize[3]
        
        central = tf.expand_dims(tf.expand_dims(central, 3), 4)
                                                
        #create and reparameterize light field grid
        b_vals = tf.to_float(tf.range(b_sz))
        v_vals = tf.to_float(tf.range(v_sz)) - tf.to_float(v_sz)/2.0
        u_vals = tf.to_float(tf.range(u_sz)) - tf.to_float(u_sz)/2.0
        y_vals = tf.to_float(tf.range(y_sz))
        x_vals = tf.to_float(tf.range(x_sz))
    
        b, y, x, v, u = tf.meshgrid(b_vals, y_vals, x_vals, v_vals, u_vals, indexing='ij')
               
        #warp coordinates by ray depths
        y_t = y + v * ray_depths
        x_t = x + u * ray_depths
        
        v_r = tf.zeros_like(b)
        u_r = tf.zeros_like(b)
        
        #indices for linear interpolation
        b_1 = tf.to_int32(b)
        y_1 = tf.to_int32(tf.floor(y_t))
        y_2 = y_1 + 1
        x_1 = tf.to_int32(tf.floor(x_t))
        x_2 = x_1 + 1
        v_1 = tf.to_int32(v_r)
        u_1 = tf.to_int32(u_r)
        
        y_1 = tf.clip_by_value(y_1, 0, y_sz-1)
        y_2 = tf.clip_by_value(y_2, 0, y_sz-1)
        x_1 = tf.clip_by_value(x_1, 0, x_sz-1)
        x_2 = tf.clip_by_value(x_2, 0, x_sz-1)
        
        #assemble interpolation indices
        interp_pts_1 = tf.stack([b_1, y_1, x_1, v_1, u_1], -1)
        interp_pts_2 = tf.stack([b_1, y_2, x_1, v_1, u_1], -1)
        interp_pts_3 = tf.stack([b_1, y_1, x_2, v_1, u_1], -1)
        interp_pts_4 = tf.stack([b_1, y_2, x_2, v_1, u_1], -1)
        
        #gather light fields to be interpolated
        lf_1 = tf.gather_nd(central, interp_pts_1)
        lf_2 = tf.gather_nd(central, interp_pts_2)
        lf_3 = tf.gather_nd(central, interp_pts_3)
        lf_4 = tf.gather_nd(central, interp_pts_4)
        
        #calculate interpolation weights
        y_1_f = tf.to_float(y_1)
        x_1_f = tf.to_float(x_1)
        d_y_1 = 1.0 - (y_t - y_1_f)
        d_y_2 = 1.0 - d_y_1
        d_x_1 = 1.0 - (x_t - x_1_f)
        d_x_2 = 1.0 - d_x_1
        
        w1 = d_y_1 * d_x_1
        w2 = d_y_2 * d_x_1
        w3 = d_y_1 * d_x_2
        w4 = d_y_2 * d_x_2
        
        lf = tf.add_n([w1*lf_1, w2*lf_2, w3*lf_3, w4*lf_4])
                        
    return lf

In [ ]:
#resample ray depths for depth consistency regularization

def transform_ray_depths(ray_depths, u_step, v_step, lfsize):
    with tf.variable_scope('transform_ray_depths') as scope:
        b_sz = tf.shape(ray_depths)[0]
        y_sz = tf.shape(ray_depths)[1]
        x_sz = tf.shape(ray_depths)[2]
        u_sz = lfsize[2]
        v_sz = lfsize[3]
                                                        
        #create and reparameterize light field grid
        b_vals = tf.to_float(tf.range(b_sz))
        v_vals = tf.to_float(tf.range(v_sz)) - tf.to_float(v_sz)/2.0
        u_vals = tf.to_float(tf.range(u_sz)) - tf.to_float(u_sz)/2.0
        y_vals = tf.to_float(tf.range(y_sz))
        x_vals = tf.to_float(tf.range(x_sz))
    
        b, y, x, v, u = tf.meshgrid(b_vals, y_vals, x_vals, v_vals, u_vals, indexing='ij')
               
        #warp coordinates by ray depths
        y_t = y + v_step * ray_depths
        x_t = x + u_step * ray_depths
        
        v_t = v - v_step + tf.to_float(v_sz)/2.0
        u_t = u - u_step + tf.to_float(u_sz)/2.0
        
        #indices for linear interpolation
        b_1 = tf.to_int32(b)
        y_1 = tf.to_int32(tf.floor(y_t))
        y_2 = y_1 + 1
        x_1 = tf.to_int32(tf.floor(x_t))
        x_2 = x_1 + 1
        v_1 = tf.to_int32(v_t)
        u_1 = tf.to_int32(u_t)
        
        y_1 = tf.clip_by_value(y_1, 0, y_sz-1)
        y_2 = tf.clip_by_value(y_2, 0, y_sz-1)
        x_1 = tf.clip_by_value(x_1, 0, x_sz-1)
        x_2 = tf.clip_by_value(x_2, 0, x_sz-1)
        v_1 = tf.clip_by_value(v_1, 0, v_sz-1)
        u_1 = tf.clip_by_value(u_1, 0, u_sz-1)
        
        #assemble interpolation indices
        interp_pts_1 = tf.stack([b_1, y_1, x_1, v_1, u_1], -1)
        interp_pts_2 = tf.stack([b_1, y_2, x_1, v_1, u_1], -1)
        interp_pts_3 = tf.stack([b_1, y_1, x_2, v_1, u_1], -1)
        interp_pts_4 = tf.stack([b_1, y_2, x_2, v_1, u_1], -1)
        
        #gather light fields to be interpolated
        lf_1 = tf.gather_nd(ray_depths, interp_pts_1)
        lf_2 = tf.gather_nd(ray_depths, interp_pts_2)
        lf_3 = tf.gather_nd(ray_depths, interp_pts_3)
        lf_4 = tf.gather_nd(ray_depths, interp_pts_4)
        
        #calculate interpolation weights
        y_1_f = tf.to_float(y_1)
        x_1_f = tf.to_float(x_1)
        d_y_1 = 1.0 - (y_t - y_1_f)
        d_y_2 = 1.0 - d_y_1
        d_x_1 = 1.0 - (x_t - x_1_f)
        d_x_2 = 1.0 - d_x_1
        
        w1 = d_y_1 * d_x_1
        w2 = d_y_2 * d_x_1
        w3 = d_y_1 * d_x_2
        w4 = d_y_2 * d_x_2
        
        lf = tf.add_n([w1*lf_1, w2*lf_2, w3*lf_3, w4*lf_4])
                        
    return lf

In [ ]:
#loss to encourage consistency of ray depths corresponding to same scene point

def depth_consistency_loss(x, lfsize):
    x_u = transform_ray_depths(x, 1.0, 0.0, lfsize)
    x_v = transform_ray_depths(x, 0.0, 1.0, lfsize)
    x_uv = transform_ray_depths(x, 1.0, 1.0, lfsize)
    d1 = (x[:,:,:,1:,1:]-x_u[:,:,:,1:,1:])
    d2 = (x[:,:,:,1:,1:]-x_v[:,:,:,1:,1:])
    d3 = (x[:,:,:,1:,1:]-x_uv[:,:,:,1:,1:])
    l1 = tf.reduce_mean(tf.abs(d1)+tf.abs(d2)+tf.abs(d3))
    return l1

In [ ]:
#normalize to between -1 and 1, given input between 0 and 1

def normalize_lf(lf):
    return 2.0*(lf-0.5)

def to_img(img):
    img_ = np.reshape(np.transpose(img, [0, 3, 4, 1, 2, 5]), [64, lfsize[0], lfsize[1], 3])
    img__ = img_ - np.min(img_)
    return img__ / np.max(img__)

def save_img(img,out_dir):
    for i in range(64):
        if (i // 8) % 2 == 0:
            j = i
        else:
            j = ((i // 8) + 1) * 8 - (i % 8) - 1
        PILImage.fromarray((img[j] * 255).astype(np.uint8)).save(f"{out_dir}/%03d.png" % i)
    

# Inference

In [ ]:
def build_graph():
    input_image = tf.placeholder(shape=[1] + lfsize[:2] + [3], dtype=tf.float32)
    _, _, y = forward_model(input_image, lfsize, disp_mult)
    return input_image, y

input_image_ph, y_ph = build_graph()

In [ ]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.restore(sess,checkpoint)
    for filename in os.listdir(input_dir):
        input_image_path = os.path.join(input_dir,filename)
        input_image = normalize_lf(np.array(PILImage.open(input_image_path).convert("RGB")).astype(np.float32))
        [y] = sess.run([y_ph], feed_dict={input_image_ph:input_image[None]})
        if not os.path.exists(outdir + filename[:-4]):
            os.makedirs(outdir + filename[:-4])
        save_img(to_img(y),out_dir=outdir + filename[:-4])
        print("test image: %s"%filename)
